In [63]:
import random
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import pandas as pd
#import imageio
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt
%matplotlib inline
import math

import optuna#载入optuna优化包

device = torch.device('cpu')

SEED = 1234
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED) 
torch.backends.cudnn.deterministic = True

xtrain = torch.unsqueeze(torch.linspace(-1, 1, 21), dim=1).to(device)
ytrain = (torch.sin(math.pi*xtrain) + 0.2*torch.randn(xtrain.size())).to(device)  

#测试数据
SEED = 666
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED) 
torch.backends.cudnn.deterministic = True

ytest = (torch.sin(math.pi*xtrain) + 0.2*torch.randn(xtrain.size())).to(device)  

In [64]:
class GEU(nn.Module):  # 激活函数含噪声参数
    def __init__(self, trial):
        super().__init__()
        self.sigma = trial.suggest_float("sigma", 0.1, 10.0)  # 设置 sigma 为可学习参数
        
    def forward(self, input):
        temp = 0.5 + torch.erf(input / self.sigma / math.sqrt(2)) / 2  # GEU 的前向传播
        return temp

class Net(torch.nn.Module):  # 定义网络
    def __init__(self, n_feature, n_hidden1, n_output, trial):
        super(Net, self).__init__()
        self.hidden1 = torch.nn.Linear(n_feature, n_hidden1)  # 全连接层
        self.custom1 = GEU(trial)  # 自定义激活函数层
        self.predict = torch.nn.Linear(n_hidden1, n_output)   # 输出层

    def forward(self, x):
        x = self.hidden1(x)
        x = self.custom1(x)
        x = self.predict(x)  # 前向传播过程
        return x

def objective(trial):
    # 在 objective 函数中创建网络实例并将 trial 对象传递给构造函数
    net = Net(n_feature=1, n_hidden1=10, n_output=1, trial=trial).to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=0.01, betas=(0.96, 0.96))
    loss_func = torch.nn.MSELoss().to(device)  # MSE损失函数
    epoch = 20000
    
    # 定义变量来存储最小损失和对应的最优参数
    min_loss = float('inf')
    best_params = None
    best_weights = None
    
    for t in range(epoch):
        prediction = net(x)  # 前向过程
        loss = loss_func(prediction, y) # 计算mse
 
        ##后向传播更新
        optimizer.zero_grad()   # clear gradients for next train
        loss.backward(retain_graph=True)    # 反向传播
        optimizer.step()        # 更新参数
        
        # 检查当前损失是否是最小损失，并更新最优参数
        if loss < min_loss:
            min_loss = loss
            best_params = trial.params.copy()
            best_weights = net.state_dict()

    # 保存最优权重到文件中
    torch.save(best_weights, 'D:/贝叶斯优化与噪声注入/best_weights.pth')
    
    # 将最优参数和损失存储到合适的位置
    study.set_user_attr('best_params', best_params)  # 存储最优参数到Study对象中
    study.set_user_attr('best_loss', min_loss.item())  # 存储最小损失到Study对象中
    
    output = net(xtrain)  
    # test误差
    losstest = loss_func(ytest, output)
    return losstest.item()


In [65]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

for trial in study.trials:
    print(f"Trial {trial.number}: Value = {trial.value}")

# 输出结果
print("Best trial:")
print("  Value: ", study.best_trial.value)
print("  Params: ")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

[I 2024-03-16 14:03:41,619] A new study created in memory with name: no-name-2fbeed06-5bdc-40fc-9bdd-32aaa4433995
[I 2024-03-16 14:03:53,888] Trial 0 finished with value: 0.05645465850830078 and parameters: {'sigma': 2.1705789003151965}. Best is trial 0 with value: 0.05645465850830078.
[I 2024-03-16 14:04:05,644] Trial 1 finished with value: 0.03544168174266815 and parameters: {'sigma': 5.441740503133442}. Best is trial 1 with value: 0.03544168174266815.
[I 2024-03-16 14:04:17,979] Trial 2 finished with value: 0.03436843305826187 and parameters: {'sigma': 8.854504579838105}. Best is trial 2 with value: 0.03436843305826187.
[I 2024-03-16 14:04:29,777] Trial 3 finished with value: 0.03444141894578934 and parameters: {'sigma': 7.2383555733740925}. Best is trial 2 with value: 0.03436843305826187.
[I 2024-03-16 14:04:42,128] Trial 4 finished with value: 0.036217451095581055 and parameters: {'sigma': 4.825034479859365}. Best is trial 2 with value: 0.03436843305826187.
[I 2024-03-16 14:04:54,

Trial 0: Value = 0.05645465850830078
Trial 1: Value = 0.03544168174266815
Trial 2: Value = 0.03436843305826187
Trial 3: Value = 0.03444141894578934
Trial 4: Value = 0.036217451095581055
Trial 5: Value = 0.04815973713994026
Trial 6: Value = 0.05638769641518593
Trial 7: Value = 0.07189932465553284
Trial 8: Value = 0.034378547221422195
Trial 9: Value = 0.034411776810884476
Trial 10: Value = 0.034608811140060425
Trial 11: Value = 0.03641422465443611
Trial 12: Value = 0.03475179150700569
Trial 13: Value = 0.03481900691986084
Trial 14: Value = 0.034249577671289444
Trial 15: Value = 0.03422980755567551
Trial 16: Value = 0.03471489995718002
Trial 17: Value = 0.03488403558731079
Trial 18: Value = 0.034814100712537766
Trial 19: Value = 0.034241944551467896
Trial 20: Value = 0.04786652699112892
Trial 21: Value = 0.034496113657951355
Trial 22: Value = 0.03452753275632858
Trial 23: Value = 0.034836169332265854
Trial 24: Value = 0.03452402725815773
Trial 25: Value = 0.034760840237140656
Trial 26: Va

In [67]:
#在求出最优sigma的基础上运行网络
class GEU(nn.Module):  # 激活函数含噪声参数
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma  # 直接接收 sigma 参数
        
    def forward(self, input):
        temp = 0.5 + torch.erf(input / self.sigma / math.sqrt(2)) / 2  # GEU 的前向传播
        return temp

class Net(torch.nn.Module):  # 定义网络
    def __init__(self, n_feature, n_hidden1, n_output, sigma):
        super(Net, self).__init__()
        self.hidden1 = torch.nn.Linear(n_feature, n_hidden1)  # 全连接层
        self.custom1 = GEU(sigma)  # 自定义激活函数层
        self.predict = torch.nn.Linear(n_hidden1, n_output)   # 输出层

    def forward(self, x):
        x = self.hidden1(x)
        x = self.custom1(x)
        x = self.predict(x)  # 前向传播过程
        return x

# 获取最佳试验参数
best_trial = study.best_trial

best_weights = torch.load('D:/贝叶斯优化与噪声注入/best_weights.pth')

# 使用最优试验参数和加载的权重参数重新初始化网络
best_sigma = best_trial.params['sigma']
best_net = Net(n_feature=1, n_hidden1=10, n_output=1, sigma=best_sigma)
best_net.load_state_dict(best_weights)

# 获取网络的输出
inputx= torch.unsqueeze(torch.linspace(-1, 1, 201), dim=1).to(device)
output = best_net(inputx)

import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams ['font.serif'] = ['Times New Roman']
mpl.rcParams['figure.dpi'] = 800
mpl.rcParams['axes.unicode_minus']=False  # #用来正常显示负号
#plt.style.use(['science','ieee'])#如果找不到science plot包则载入这句话
fig, ax = plt.subplots(figsize=(5,4)) 
plt.cla()
plt.tick_params(labelsize=15)
ax.set_xlabel('x', fontsize=15)
ax.set_ylabel('y', fontsize=15, labelpad = -1)
ax.set_xlim(-1.1, 1.1)
ax.set_ylim(-1.5, 1.5)

# 训练集橘色的散点
ax.scatter(xtrain.data.numpy(), ytrain.data.numpy(), color="orange")
 # 非常密的输入和网络输出
ax.plot(inputx.data.numpy(), output.data.numpy(), 'g-', lw=2)
#真实信号
signal = torch.sin(math.pi*inputx)
ax.plot(inputx.data.numpy(), signal.data.numpy(), 'r--', lw=1,label='signal')

AttributeError: 'float' object has no attribute 'suggest_float'